https://www.kaggle.com/c/quora-insincere-questions-classification/data

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.text import *

In [3]:
path = Path('/Users/nikhil/workspace/fastaiv1/my_data/quora')

In [4]:
df = pd.read_csv(path/'train.csv')

In [5]:
def label(row):
    return 'positive' if row.target else 'negative'

df['label'] = df.apply(lambda row: label(row), axis=1)
df = df.rename(columns = {"question_text": "text"})
df = df.drop(['qid', 'target'], axis=1)

In [6]:
import random
# assign true value with probability of 0.8
df['is_valid'] = df.apply(lambda x:0.8 > random.random(), axis=1)

#len(df) == len(df.qid.unique()) #1306122

In [7]:
len_df = int(len(df)*.8)
train_df, valid_df = df[:len_df], df[len_df:]
# for cpu local
train_df = train_df[:int(len(train_df)*.01)]
# valid_df = valid_df[:int(len(valid_df)*.01)]
train_df.head()

,text,label,is_valid
0,How did Quebec nationalists see their province...,negative,True
1,"Do you have an adopted dog, how would you enco...",negative,True
2,Why does velocity affect time? Does velocity a...,negative,False
3,How did Otto von Guericke used the Magdeburg h...,negative,True
4,Can I convert montra helicon D to a mountain b...,negative,True


In [8]:
train_df.to_csv(path/'texts.csv', index=False, columns=['label', 'text', 'is_valid'])
# valid_df.to_csv(path/'valid.csv', index=False, columns=['label', 'text', 'is_valid'])

In [9]:
# data_lm = TextDataBunch.from_csv(path, train='texts', valid='valid')
data_lm = TextDataBunch.from_csv(path, 'texts.csv')

In [10]:
data_lm.save()

In [11]:
type(data_lm)

fastai.text.data.TextDataBunch

In [19]:
# data = TextDataBunch.load(path)
data = TextClasDataBunch.load(path)
data.show_batch()

text,label
"xxfld 1 can i get into xxup mit with a 29 xxup act ( 30 e , 33 m , 26 r , 28 s ) ? i xxunk to the us from korea during 10th grade ( here on a green card ) . my xxup xxunk is xxunk . my xxunk and xxunk are great ( some b 's , but mostly a 's ) . my xxunk 's and passion are also great . xxfld 2 true",negative
xxfld 1 can the xxunk interest formula p = l [ c ( 1 + c ) n ] / [ ( 1 + c ) n - 1 ] where p is the fixed monthly payment required to fully xxunk a loan of l dollars over a term of n months at a monthly interest rate of c be xxunk to solve for n ? xxfld 2 true,negative
"xxfld 1 is there a doctor who did his / her xxup mbbs from a not - so - good government medical college but did his / her xxunk ( md , xxunk , xxunk ) from a good and xxunk medical xxunk ? if yes , then how did he / she xxunk to get into the good college later in life ? xxfld 2 false",negative
"xxfld 1 why does xxunk 's last theorem xxunk immediately the non - xxunk of such xxunk real xxunk xxunk numbers of this form ( [ math ] \ xxunk } [ / math ] ) , where [ math ] ( p xxunk 2 , q ) [ / math ] are prime numbers in science and mathematics ? xxfld 2 true",negative
"xxfld 1 what does it mean if xxunk sends you his xxunk and you can see a woman behind him , she is far too see her clearly and you ca n't say anything xxunk but she ( or a part of her body : ) ) ) appears almost on every xxunk he sends ? to make jealous ? xxfld 2 true",negative
"xxfld 1 how can i not be xxunk by my parents so often and stay xxunk ? my dad is very xxunk , can be rude and xxunk . my mom acts extremely happy ( does n't feel genuine ) , xxunk , or very angry . i love them , but am so different from them . xxfld 2 false",negative
"xxfld 1 offers from xxunk ( xxunk , sales development xxunk ) and xxunk ( hong kong , online ads account manager ) , and interview xxunk from xxup sap ( hong kong ) and xxup sas ( taiwan ) sales academy . which one should i choose for a better sales career in tech ? xxfld 2 false",negative
"xxfld 1 "" the company a , which is company b 's xxunk , xxunk makes the design for the company xxunk 's premium product . thus the company b xxunk always has the capabilities to make premium designs "" how do you understand it ? can ' thus ' link the 2 xxunk ? xxfld 2 true",negative
"xxfld 1 how do i change false beliefs about myself ? i feel ugly and awkward and i know i 'm not . i hide from people , like i was the xxunk of xxunk - xxunk , while i am attractive and quite articulate . how do i free myself from this xxunk ? xxfld 2 true",negative
"xxfld 1 how have we xxunk so much debt even though we all pay taxes ? is it from the current debt that individual people are in , therefore they pay back on their own ? and if not , will this eventually create a xxunk / war with other countries ? xxfld 2 true",negative


In [20]:
data.train_ds.vocab.itos, data.train_ds[0]

(['xxunk',
  'xxpad',
  'xxfld',
  '?',
  '1',
  '2',
  'true',
  'the',
  'what',
  'is',
  'a',
  'to',
  'in',
  'of',
  'i',
  'how',
  'do',
  'false',
  'are',
  'and',
  ',',
  'for',
  'you',
  'xxup',
  'can',
  'why',
  'it',
  'my',
  'that',
  '.',
  'with',
  'if',
  'be',
  'on',
  'does',
  'or',
  'have',
  '"',
  'from',
  "'s",
  'should',
  'which',
  'get',
  'your',
  'would',
  'best',
  'when',
  'an',
  '-',
  'some',
  'as',
  'people',
  'there',
  "n't",
  ')',
  'who',
  '(',
  'not',
  'did',
  'like',
  'will',
  'at',
  'about',
  'they',
  'was',
  'by',
  'so',
  'their',
  'we',
  'any',
  'good',
  '/',
  'me',
  'india',
  'but',
  'one',
  'where',
  'has',
  'more',
  'think',
  'many',
  'after',
  'most',
  'other',
  'make',
  'someone',
  'this',
  'use',
  'all',
  'time',
  'them',
  'between',
  'being',
  'out',
  'much',
  'life',
  'am',
  'than',
  'he',
  'up',
  "'",
  'into',
  'were',
  'way',
  'want',
  'ever',
  'just',
  'work',


In [22]:
data = (SplitData.from_csv(path, 'texts.csv', input_cols=1, label_cols=0, valid_col=2)
        .datasets(TextDataset)
        .tokenize() #can specify custom arguments for tokenization here
        .numericalize() #can specify custom arguments for numericalization here
        .databunch(TextDataBunch))

In [30]:
doc(SplitData)

In [23]:
learn = language_model_learner(data_lm, drop_mult=0.3, pretrained_model=URLs.WT103)

In [24]:
learn.lr_find()

LR Finder complete, type {learner_name}.recorder.plot() to see the graph.


TypeError: forward() takes 2 positional arguments but 11 were given